In [1]:
import random
import numpy as np
import requests

In [52]:
class Community:
    
    def __init__(self, totalPopulation=100, row_size=8, mutationRate=0.05):
        self.totalPopulation = totalPopulation
        self.row_size = row_size
        self.mutationRate = mutationRate
        self.population_array = self.initial_population()
        self.fitness_score()
        self.perfect_config = np.zeros((1,1))
    
    def initial_population(self):
        # generate an array of configurations represented as arrays
        # expect shape as (totalPopulation, row_size)
        
        def fill_position(a):
            return np.apply_along_axis(lambda x: random.randint(0,self.row_size-1), 0, population_array)

        population_array = np.zeros((self.totalPopulation,self.row_size))
        population_array = np.apply_along_axis(fill_position, 1, population_array)
        return population_array
    
    def fitness_score(self):
        # generate an array of shape (totalPopulation, 1) where each number corresponds
        # with a configuration in population array
        # the highest possible score is the length of the configuration times 2
        # a point for each unique value and a point for each non diagonal sharing numbers
        
        def diagonal_check(a):
            passing = 0
            
            for i in range(len(a)):
                b = np.copy(a[i:])
                
                for j in range(i, len(a)):
                    v = a[i]
                    b[j-i] = np.absolute(a[j]-v) - (j-i)
                unique, counts = np.unique(b, return_counts=True)

                if counts[np.where(unique==0)][0] == 1:
                    passing += 1
            
            return passing
    
        
        self.score = np.apply_along_axis(lambda x: len(np.unique(x)), 1, self.population_array) +\
                     np.apply_along_axis(diagonal_check, 1, self.population_array)
    
    def get_score(self):
        return self.score
    
    def crossover_mutation(self):
        # selection, crossover, and mutation in one function

        def parental_selection_and_crossover(a):
            # choose parents by weighted average in their scores
            # choose a random partition to crossover
            parents = np.array(random.choices(self.population_array, weights=self.score, k=2))
            crossOver = random.randint(0, self.row_size-1)
            return np.concatenate((parents[0,:crossOver], parents[1,crossOver:]))

        def mutation(a):
            # randomly select an index to mutate if chosen to mutate
            # each configuration was extended by a random float from 0-1
            # so it uses the last indexed number to determine whether it mutates
            # will drop the last index after applying this function
            
            if a[-1]<self.mutationRate:
                a[random.randint(0, self.row_size-1)]=random.randint(0, self.row_size-1)
                
            return a
       
        queuing = np.zeros((self.totalPopulation,1))
        developing_population = np.apply_along_axis(parental_selection_and_crossover, 1, queuing)
        mutation_chance = np.concatenate((developing_population, np.random.uniform(size=(self.totalPopulation,1))), axis=1)
        new_population = np.apply_along_axis(mutation, 1, mutation_chance)
        self.population_array = np.delete(new_population, -1, 1)
    
    def find_perfect_config(self):
        # should be used only when the a configuation has all unique numbers
        # otherwise send a dummy response
        try:
            self.perfect_config = self.population_array[np.where(self.score==self.row_size*2)[0][0]]
        except IndexError:
            self.perfect_config = np.zeros(1)
            
    def get_perfect_config(self):
        # return response in string format with a space between each number
        return ' '.join(list(map(lambda x:str(int(x)), list(self.perfect_config))))
    

In [83]:
def main():
    
    totalPopulation = 500
    row_size = 8
    mutationRate = 0.10
    
    the_community = Community(totalPopulation, row_size, mutationRate)

    endless_loop = True
    counter = 0

    while endless_loop:
        counter+=1
        if counter%100==0:
            print(f'Generation: {counter} {np.mean(the_community.get_score())}')
        
        if max(the_community.get_score())==row_size*2:
            the_community.find_perfect_config()
            print(the_community.get_perfect_config())
            print(f'Generation: {counter}')
            #url='https://lf8q0kx152.execute-api.us-east-2.amazonaws.com/default/computeFitnessScore'
            #x=requests.post(url,json={"qconfig":the_community.get_perfect_config(),"userID":843868,"githubLink":"https://github.com/cdinhphil/datasummit_python_challenge1"})
            #print(x.text)

            
            break

        the_community.crossover_mutation()
        the_community.fitness_score()

In [85]:
main()

4 2 0 5 7 1 3 6
Generation: 51


In [86]:
%timeit main()

2 4 6 0 3 1 7 5
Generation: 67
4 7 3 0 6 1 5 2
Generation: 14
0 6 3 5 7 1 4 2
Generation: 37
1 5 0 6 3 7 2 4
Generation: 7
Generation: 100 11.07
4 2 7 3 6 0 5 1
Generation: 149
6 4 2 0 5 7 1 3
Generation: 54
5 1 6 0 3 7 4 2
Generation: 56
Generation: 100 11.646
Generation: 200 12.836
Generation: 300 12.888
Generation: 400 13.046
1 4 6 0 2 7 5 3
Generation: 441
The slowest run took 63.30 times longer than the fastest. This could mean that an intermediate result is being cached.
15.3 s ± 19.9 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


****************** Unit Testing *************************

In [37]:
import unittest

class TestProcessing(unittest.TestCase):
    
    def setUp(self):
        self.community = Community(100, 8, 0.05)
        
    def test_initial_population(self):
        self.assertEqual(self.community.population_array.shape, (100,8))
        n = set(np.arange(8))
        
        for i in list(set(self.community.population_array.flatten())):
            self.assertIn(i, n)
            
    def test_fitness_score(self):
        self.assertLessEqual(max(self.community.score), 8)
        self.assertGreaterEqual(min(self.community.score),0)
        
    def test_get_score(self):
        self.assertEqual(self.community.population_array.shape, (100,8))
        n = set(np.arange(8))
        
        for i in list(set(self.community.get_score())):
            self.assertIn(i, n)
            
    def test_crossover_mutation(self):
        old_generation = self.community.population_array
        self.community.crossover_mutation()
        self.assertRaises(AssertionError, np.testing.assert_array_equal, old_generation, self.community.population_array)
        self.assertEqual(self.community.population_array.shape, (100,8))
        n = set(np.arange(8))
        
        for i in list(set(self.community.population_array.flatten())):
            self.assertIn(i, n)
            
    def test_find_perfect_config(self):
        self.community.population_array = np.zeros((100,8))
        self.community.population_array[0] = np.arange(8)
        self.community.fitness_score()
        self.community.find_perfect_config()
        np.testing.assert_array_equal(self.community.perfect_config, np.arange(8))
        
    def test_get_perfect_config(self):
        self.community.perfect_config = np.arange(8)
        self.assertEqual(self.community.get_perfect_config(), '0 1 2 3 4 5 6 7')
        

In [38]:
a = TestProcessing()
a.setUp()
a.test_initial_population()
a.test_fitness_score()
a.test_get_score()
a.test_crossover_mutation()
a.test_find_perfect_config()
a.test_get_perfect_config()

In [71]:
a = np.array([4, 2, 0, 6, 1, 7, 5, 3])
passing = 0
for i in range(len(a)):
    b = np.copy(a[i:])
    for j in range(i, len(a)):
        v = a[i]
        b[j-i] = np.absolute(a[j]-v) - (j-i)
    unique, counts = np.unique(b, return_counts=True)
    
    if counts[np.where(unique==0)][0] == 1:
        passing += 1
print(passing)

b = np.zeros((8,8))
for k in range(8):
    b[k][a[k]] = 1
print(b)

8
[[0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]]
